In [ ]:
import pandas as pd
from sqlalchemy import create_engine

user = 'project'
password = 'myproject'
host = 'localhost'
database = 'stat-uz'

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

csv_file_path = 'main_df.csv'
df = pd.read_csv(csv_file_path)

df.to_sql('uz-data', con=engine, if_exists='replace', index=False)